# IG360 QA - Capture Post Data
*Purpose:* Capture key data from all posts listed in an IG360 full profile (FP) file
<ul>
    <li>Location</li>
    <li>Like stats</li>
    <li>Raw post and comment data</li>
    <li>Full like list</li>
</ul>
<br>
<br>
Import libraries, set script configurations

In [1]:
# import IG360 data
from IG360.ig360_scrape import *

# import other key libraries
import pandas as pd
import datetime
import json
import re
from pathlib import Path

# set IG login and password (turned off by default; using config.ini)
#ig_login = "xx"
#ig_pass = "xx"

# set input / output folder
txt_dir_input = '_output/'
dir_input = Path(txt_dir_input)
txt_dir_output = '_output/'
dir_output = Path(txt_dir_output)

# locate configuration file
config_file_path = 'config.ini'
config = init_config(config_file_path)

# set source FP file and processing parameters
src_fp_file = 'fp_benchesof_instagram_202004081150.json'

Load Full Profile file and capture list of posts to process

In [2]:
# load full profile file
with open(dir_input/src_fp_file) as fp:
    profile = json.load(fp)

#get target user
tgt_user = profile['user_name']

# get list of posts to process
tgt_posts = [i.split('/')[-2] for i in profile['recent_posts']]

Initialize scraper and login

In [3]:
#scraper = IG360Scrape(config, user_login, user_password)
scraper = IG360Scrape(config)
scraper.login()

User: 


Scrape Target Posts

In [4]:
first_element = True
post_data = []
for tgt_post in tgt_posts:
    scraper.scrape_post(tgt_post)
    if first_element:
        post_data = [scraper.record_post]
        first_element = False
    else:
        post_data.append([scraper.record_post])

Process Final Ouput; Export to JSON

In [5]:
# compile final data dictionary
dt_run = datetime.datetime.now()
dict_final = {
    "run_datetime" : dt_run.strftime('%Y-%m-%dT%H:%M:%SZ'),
    "ig_record_type" : "ps",
    "code_version" : scraper.code_version,
    "fp_source" : src_fp_file,
    "posts" : post_data
}

# output as JSON
fn_out = re.sub('[\W]','',dict_final["ig_record_type"]) + '_' + tgt_user + '_' \
    + str(dt_run.year) + str(dt_run.month).zfill(2) + str(dt_run.day).zfill(2) + str(dt_run.hour).zfill(2) + str(dt_run.minute).zfill(2) + '.json'
fn_out_full = dir_output / fn_out
with open(fn_out_full, 'w') as json_file:
    json.dump(dict_final, json_file)
